1) The provided data (link above) contains various details and attributes associated with used cars. The
target variable, which is the central focus of analysis, is the price of the used cars, and it is measured in
lakhs. The data in this dataset is tabular, with rows and columns, where each row represents a specific
used car listing, and each column represents a particular attribute or feature of these cars. Features are
Make and model of the car, Location or city of sale, Year of manufacture, Mileage, Odometer
(kilometers driven), Fuel type (petrol or diesel), Transmission type (manual or automatic), Number of owners, Engine displacement, Engine horsepower, Number of seats, and Price when the car was new.

Import & Load Data

In [27]:
import pandas as pd
import numpy as np
df = pd.read_csv('train.csv')
print(df.shape)
df.head()


(5847, 14)


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


**a) Look for the missing values in all the columns and either impute them (replace with mean,
median, or mode) or drop them. Justify your action for this task.**

Step-1 Checking the missing values in all columns


In [28]:
df.isnull().sum()


,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,36


New_Price has **5032/5847 ≈ 86%** missing too many to impute reliably.

Mileage, Engine, Power, Seats have relatively few missing we can impute.

Step-2 Remove Units for numeric columns to impute

In [29]:
df_clean = df.copy()

def extract_number(x):
    if pd.isnull(x):
        return np.nan
    s = str(x).split()
    try:
        return float(s[0])
    except ValueError:
        return np.nan

for col in ['Mileage', 'Engine', 'Power', 'New_Price']:
    df_clean[col] = df_clean[col].apply(extract_number)

df_clean[['Mileage', 'Engine', 'Power', 'New_Price']].head()


,Mileage,Engine,Power,New_Price
0,19.67,1582.0,126.20,NaN
1,13.00,1199.0,88.70,8.61
2,20.77,1248.0,88.76,NaN
3,15.20,1968.0,140.80,NaN
4,23.08,1461.0,63.10,NaN


Step-3 Drop New_Price Impute Others with Median

In [30]:
df_imputed = df_clean.drop(columns=['New_Price'])
for col in ['Mileage', 'Engine', 'Power', 'Seats']:
    median_val = df_imputed[col].median()
    df_imputed[col].fillna(median_val, inplace=True)
df_imputed.isnull().sum()


/tmp/ipython-input-511131743.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_imputed[col].fillna(median_val, inplace=True)


,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,0
Engine,0


I dropped New_Price because >80% missing, and for Mileage, Engine, Power, Seats i use Median. which keeps distribution reasonable.

**b) Remove the units from some of the attributes and only keep the numerical values (for
example remove kmpl from “Mileage”, CC from “Engine”, bhp from “Power”, and lakh from
“New_price”).**

In [31]:
df_unit_removed = df.copy()

def extract_number(x):
    if pd.isnull(x):
        return np.nan
    s = str(x).split()
    try:
        return float(s[0])
    except ValueError:
        return np.nan

# Remove units
df_unit_removed['Mileage']   = df_unit_removed['Mileage'].apply(extract_number)
df_unit_removed['Engine']    = df_unit_removed['Engine'].apply(extract_number)
df_unit_removed['Power']     = df_unit_removed['Power'].apply(extract_number)
df_unit_removed['New_Price'] = df_unit_removed['New_Price'].apply(extract_number)

df_unit_removed[['Mileage','Engine','Power','New_Price']].head()


,Mileage,Engine,Power,New_Price
0,19.67,1582.0,126.20,NaN
1,13.00,1199.0,88.70,8.61
2,20.77,1248.0,88.76,NaN
3,15.20,1968.0,140.80,NaN
4,23.08,1461.0,63.10,NaN


**C) Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot
encoded value.**

One-Hot Encode Fuel_Type and Transmission

In [32]:
df_encoded = pd.get_dummies(
    df_imputed,
    columns=['Fuel_Type', 'Transmission'],
    drop_first=False   # keep all categories
)

df_encoded.head()


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,12.50,True,False,False,False,True
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,4.50,False,False,True,False,True
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,True,False,False,False,True
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,17.74,True,False,False,True,False
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,3.50,True,False,False,False,True


Expected new columns included

Fuel_Type_Diesel, Fuel_Type_Petrol, Fuel_Type_Electric,  Transmission_Manual, Transmission_Automatic

**d) Create one more feature and add this column to the dataset (you can use mutate function in
R for this). For example, you can calculate the current age of the car by subtracting “Year” value
from the current year.**

Creating a New Feature: Car Age

using Car_Age = current_year – Year.

In [33]:
import datetime

current_year = datetime.datetime.now().year  # e.g., 2025
df_encoded['Car_Age'] = current_year - df_encoded['Year']

df_encoded[['Year', 'Car_Age']].head()


,Year,Car_Age
0,2015,10
1,2011,14
2,2012,13
3,2013,12
4,2013,12


**e) Perform select, filter, rename, mutate, arrange and summarize with group by operations (or
their equivalent operations in python) on this dataset.**

I use pandas equivalents

**1) select - for choosing specific columns**

In [34]:
selected = df_encoded[['Name', 'Year', 'Kilometers_Driven', 'Price']]
selected.head()


,Name,Year,Kilometers_Driven,Price
0,Hyundai Creta 1.6 CRDi SX Option,2015,41000,12.50
1,Honda Jazz V,2011,46000,4.50
2,Maruti Ertiga VDI,2012,87000,6.00
3,Audi A4 New 2.0 TDI Multitronic,2013,40670,17.74
4,Nissan Micra Diesel XV,2013,86999,3.50


**2) filter – Rows Matching a Condition**

Example: cars older than 10 years and price < 5 lakhs:

In [35]:
filtered = df_encoded[
    (df_encoded['Car_Age'] > 10) & (df_encoded['Price'] < 5)
][['Name', 'Year', 'Car_Age', 'Price']]

filtered.head()


,Name,Year,Car_Age,Price
1,Honda Jazz V,2011,14,4.50
4,Nissan Micra Diesel XV,2013,12,3.50
7,Tata Indica Vista Quadrajet LS,2012,13,1.95
9,Honda City 1.5 V AT Sunroof,2012,13,4.49
21,Hyundai i20 1.2 Magna,2010,15,1.87


**3) rename – Rename a Column**

Example: Kilometers_Driven to Kms_Driven

In [17]:
renamed = df_encoded.rename(columns={'Kilometers_Driven': 'Kms_Driven'})
renamed[['Name', 'Kms_Driven']].head()


,Name,Kms_Driven
0,Hyundai Creta 1.6 CRDi SX Option,41000
1,Honda Jazz V,46000
2,Maruti Ertiga VDI,87000
3,Audi A4 New 2.0 TDI Multitronic,40670
4,Nissan Micra Diesel XV,86999


**4) mutate – Create a New Derived Column**

Example: Price_per_1000km = Price / (Kilometers_Driven / 1000)

In [16]:
df_mutated = df_encoded.copy()
df_mutated['Price_per_1000km'] = df_mutated['Price'] / (df_mutated['Kilometers_Driven'] / 1000)

df_mutated[['Name', 'Kilometers_Driven', 'Price', 'Price_per_1000km']].head()


,Name,Kilometers_Driven,Price,Price_per_1000km
0,Hyundai Creta 1.6 CRDi SX Option,41000,12.50,0.304878
1,Honda Jazz V,46000,4.50,0.097826
2,Maruti Ertiga VDI,87000,6.00,0.068966
3,Audi A4 New 2.0 TDI Multitronic,40670,17.74,0.436194
4,Nissan Micra Diesel XV,86999,3.50,0.040230


**5) arrange – Sort Rows (Order By)**

Example: sort by Price descending:

In [15]:
arranged = df_encoded.sort_values(by='Price', ascending=False)[['Name', 'Year', 'Price']]
arranged.head()


,Name,Year,Price
3952,Land Rover Range Rover 3.0 Diesel LWB Vogue,2017,160.00
5620,Lamborghini Gallardo Coupe,2011,120.00
5752,Jaguar F Type 5.0 V8 S,2015,100.00
1457,Land Rover Range Rover Sport SE,2019,97.07
1917,BMW 7 Series 740Li,2018,93.67


**6) summarize + group_by – Aggregation by Fuel Type**

I Use the pre-encoded but still categorical version df_imputed for grouping by Fuel_Type.

In [14]:
summary = df_imputed.groupby('Fuel_Type')['Price'].agg(
    ['count', 'mean', 'min', 'max']
).reset_index()

summary


,Fuel_Type,count,mean,min,max
0,Diesel,3161,12.960686,0.60,160.0
1,Electric,2,12.875000,12.75,13.0
2,Petrol,2684,5.756688,0.44,120.0
